# Many simulations at different carrying capacities and switching speeds

With dominant paths displayed alongside their weights.

In [1]:
import pandas as pd
import numpy as np
import math
import os
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [3]:
def hist_carrying_cap(landscape, k_values=[9, 8, 7, 6], num=100, **kargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    data = []
    for k in k_values:
        param = dict(kargs)
        param['carrying_cap'] = 10**k
        param['prob_mutation'] = 10**(-(k-1))
        times = [simulate(landscape, **param)['T_f'] for i in range(num)]
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            #xbins = dict(
            #    start = 0,
            #    end = 1000,
            #    size = 10),
            #autobinx = False,
            opacity=0.75,
            name = 'K=10^{}'.format(k)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different carrying capacities'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [4]:
def hist_switching(landscapes, frequencies=[200, 100, 50], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            xbins = dict(
                size = 1),
            autobinx = False,
            opacity=0.75,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            #rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [5]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    #success_count = 0
    greedy_path = ''
    paths = {}
    times = [] # T_f's
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
        times.append(results['T_f'])
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    num_paths = sum([c > 0 for c, sc in paths.values()])
    num_successful_paths = sum([sc > 0 for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if not np.isnan(max_weight):
        dominant_path = df.iloc[df['Weight'].idxmax()]['Pathway']
    else:
        dominant_path = ""
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {num_successful_paths / num_paths}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_count': total_success_count,
            'success_rate': num_successful_paths / num_paths,
            'dominant_path': dominant_path,
            'times': times
        }

In [6]:
def table_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    success_counts = []
    success_rates = []
    weights = []
    doms = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        analysis = pathway_analysis(landscapes, **param)
        success_counts.append(analysis['success_count'])
        success_rates.append(analysis['success_rate'])
        weights.append(analysis['max_weight'])
        doms.append(analysis['dominant_path'])
    df = pd.DataFrame({
        'f': frequencies,
        'Success count': success_counts,
        'Success rate': success_rates,
        'Weight of dominant path': weights,
        'Dominant path': doms
    }).set_index('f')
    return df

In [21]:
figin = ord('a')
def box_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], k=9, num=100, **kargs):
    global figin
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    data = []
    wx = []
    wy = []
    param = dict(kargs)
    param['carrying_cap'] = 10**k
    param['prob_mutation'] = 10**(-(k-1))
    for f in frequencies:
        param['frequency'] = f
        analysis = pathway_analysis(landscapes, num=num, **param)
        times = [t for t in analysis['times'] if t != -1]
        trace = go.Box(
            #x = [f for i in range(len(times))],
            y = times,
            boxpoints = False,
            name = 'f={}'.format(f)
        )
        data.append(trace)
        if not np.isnan(analysis['max_weight']):
            wx.append('f={}'.format(f))
            wy.append(analysis['max_weight'])
    data.append(go.Scatter(
        x=wx,
        y=wy,
        name = 'Wt. of dom. path',
        yaxis = 'y2',
        marker = dict(
            color = 'rgb(0, 0, 0)'
        )
    ))
    layout = go.Layout(
        title = 'Running {} simulations on {} at K=10^{}'.format(num, name, k),
        xaxis = dict(
            title = 'Switching speed',
        ),
        yaxis = dict(
            title = 'Time to fixation'
        ),
        yaxis2 = dict(
            title = 'Weight of dominant path',
            overlaying = 'y',
            side = 'right',
        ),
        showlegend = False
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)
    pio.write_image(fig, 'fig7{}.pdf'.format(chr(figin)))
    figin += 1

In [19]:
def curves_switching(landscapes, k_values=[9, 8, 7, 6, 5, 4], frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for k in k_values:
        param = dict(kargs)
        param['carrying_cap'] = 10**k
        param['prob_mutation'] = 10**(-(k-1))
        averages = []
        for f in frequencies:
            param['frequency'] = f
            times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
            times = [t for t in times if t != -1]
            avg = (sum(times) / len(times)) if len(times) else np.nan
            averages.append(avg)
        trace = go.Scatter(
            x = frequencies,
            y = averages,
            name = 'K=10^{}'.format(k)
        )
        data.append(trace)
    layout = go.Layout(
        title = 'Running {} simulations on {} at different carrying capacities'.format(num, name),
        xaxis = dict(
            title = 'Switching frequency',
            autorange='reversed'
        ),
        yaxis = dict(
            title = 'Avg. time to fixation'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

In [20]:
figi = ord('a')
for pair in pairs:
    fig = curves_switching(pair)
    iplot(fig, show_link=False)
    pio.write_image(fig, 'fig6{}.pdf'.format(chr(figi)), width=11*95.5, height=8.5*95.5)
    figi += 1

## CTX + SAM

### K=10^9

In [1]:
#display(table_switching(pairs[0]))
box_switching(pairs[0])

NameError: name 'box_switching' is not defined

### K=10^6

In [23]:
#display(table_switching(pairs[0], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[0], k=6)

### K=10^4

In [24]:
#display(table_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[0], k=4)

## AM + AMC

### K=10^9

In [25]:
#display(table_switching(pairs[1]))
box_switching(pairs[1])

### K=10^6

In [26]:
#display(table_switching(pairs[1], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[1], k=6)

### K=10^4

In [27]:
#display(table_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[1], k=4)

## ZOX + CXM

### K=10^9

In [28]:
#display(table_switching(pairs[2]))
box_switching(pairs[2])

### K=10^6

In [29]:
#display(table_switching(pairs[2], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[2], k=6)

### K=10^4

In [30]:
#display(table_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[2], k=4)